Paper: Siamese Neural Networks for One-shot Image Recognition
http://www.cs.cmu.edu/~rsalakhu/papers/oneshot1.pdf

Images sizes: 40x40 / 75x75 / 105x105 / 120x120 / 150x150
Images in RGB / Grayscale

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import mixed_precision
import tensorflow_addons as tfa
import math
import wandb
from wandb.keras import WandbCallback
from helper_functions import plot_training
from helper_functions import create_tf_data_datasets_contrastive
from helper_functions import create_tf_data_testset_contrastive
from helper_functions import get_classification_report

# Original Model

## First Run - 30k Pairs

In [204]:
anchor_images_path = "npz_datasets/pairs_30k_224_224/anchor"
positive_images_path = "npz_datasets/pairs_30k_224_224/positive"
width, height, channels = 105, 105, 3
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=True)

In [1]:
def get_original_model(height, width, channels):

    input = keras.layers.Input(shape=(height, width, channels))

    x = keras.layers.Conv2D(64, (10,10), activation="relu",
                            kernel_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                            bias_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                            kernel_regularizer=keras.regularizers.l2(2e-4), name='Conv1')(input)
    x = keras.layers.MaxPooling2D(2,2)(x)

    x = keras.layers.Conv2D(128, (7,7), activation="relu",
                            kernel_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                            bias_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                            kernel_regularizer=keras.regularizers.l2(2e-4), name='Conv2')(x)
    x = keras.layers.MaxPooling2D(2,2)(x)

    x = keras.layers.Conv2D(128, (4,4), activation="relu",
                        kernel_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                        bias_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                        kernel_regularizer=keras.regularizers.l2(2e-4), name='Conv3')(x)
    x = keras.layers.MaxPooling2D(2,2)(x)

    x = keras.layers.Conv2D(256, (4,4), activation="relu",
                        kernel_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                        bias_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                        kernel_regularizer=keras.regularizers.l2(2e-4), name='Conv4')(x)

    x = keras.layers.Flatten()(x)
    output = keras.layers.Dense(4096, activation="sigmoid", kernel_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.2),
                           bias_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                           kernel_regularizer=keras.regularizers.l2(1e-3), name='Dense1')(x)

    model = keras.models.Model(input, output)

    return model

In [25]:
original_model = get_original_model(height,width,channels)

In [26]:
original_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 105, 105, 3)]     0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 96, 96, 64)        19264     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
Conv2 (Conv2D)               (None, 42, 42, 128)       401536    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 21, 21, 128)       0         
_________________________________________________________________
Conv3 (Conv2D)               (None, 18, 18, 128)       262272    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 9, 9, 128)         0   

In [27]:
left_input = keras.layers.Input(shape=(height, width, channels))
right_input = keras.layers.Input(shape=(height, width, channels))

encoded_l = original_model(left_input)
encoded_r = original_model(right_input)

L1_layer = keras.layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
merge_layer = L1_layer([encoded_l, encoded_r])

prediction = keras.layers.Dense(1, activation="sigmoid")(merge_layer)

original_siamese_model = keras.models.Model([left_input, right_input], outputs=prediction)

In [ ]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.001,
                         "momentum": 0.5,
                         "otimizer": "SGD",
                         "loss_function": "binary_crossentropy",
                         "epochs": 200,
                         "architecture": "Original - 30k"})

In [29]:
config = wandb.config

In [30]:
original_siamese_model.compile(loss=config.loss_function,
                               optimizer=keras.optimizers.SGD(learning_rate=config.learning_rate, momentum=config.momentum), metrics=["accuracy"])

In [32]:
def scheduler(epoch, lr):
    new_lr =  lr * 0.99
    print(f"learning rate scheduled to {new_lr}")
    return new_lr

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="Architecture_1_Checkpoints/Original_30k",
    save_weights_only=True,
    monitor="val_accuracy",
    save_best_only=True
)

In [33]:
history_original_siamese_model = original_siamese_model.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.LearningRateScheduler(scheduler),
                                                                                                                              model_checkpoint_callback, WandbCallback()])

Epoch 1/200
learning rate scheduled to 0.0009900000470224768
166/166 [==============================] - 14s 73ms/step - loss: 1509.8461 - accuracy: 0.5195 - val_loss: 1508.8473 - val_accuracy: 0.5477
Epoch 2/200
learning rate scheduled to 0.000980100086890161
166/166 [==============================] - 13s 72ms/step - loss: 1507.8730 - accuracy: 0.5517 - val_loss: 1506.8850 - val_accuracy: 0.5713
Epoch 3/200
learning rate scheduled to 0.0009702991275116801
166/166 [==============================] - 13s 72ms/step - loss: 1505.9220 - accuracy: 0.5641 - val_loss: 1504.9448 - val_accuracy: 0.5760
Epoch 4/200
learning rate scheduled to 0.0009605961316265165
166/166 [==============================] - 13s 72ms/step - loss: 1503.9926 - accuracy: 0.5570 - val_loss: 1503.0267 - val_accuracy: 0.5584
Epoch 5/200
learning rate scheduled to 0.0009509901772253215
166/166 [==============================] - 13s 72ms/step - loss: 1502.0851 - accuracy: 0.5658 - val_loss: 1501.1294 - val_accuracy: 0.5626
E

In [34]:
loss, accuracy = original_siamese_model.evaluate(val_dataset)
wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)})

42/42 [==============================] - 2s 23ms/step - loss: 1348.0594 - accuracy: 0.7954


In [206]:
anchor_images_path = "npz_datasets/test_pairs_224_224/anchor"
positive_images_path = "npz_datasets/test_pairs_224_224/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_path, positive_images_path, height, width, rgb=True)

In [195]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, original_siamese_model)

In [190]:
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()

## Second Run - 90k Pairs

In [207]:
anchor_images_path = "npz_datasets/pairs_90k_224_224/anchor"
positive_images_path = "npz_datasets/pairs_90k_224_224/positive"
width, height, channels = 105, 105, 3
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=True)

In [208]:
original_model = get_original_model(height,width,channels)
original_model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 105, 105, 3)]     0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 96, 96, 64)        19264     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
Conv2 (Conv2D)               (None, 42, 42, 128)       401536    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 21, 21, 128)       0         
_________________________________________________________________
Conv3 (Conv2D)               (None, 18, 18, 128)       262272    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 9, 9, 128)         0   

In [210]:
left_input = keras.layers.Input(shape=(height, width, channels))
right_input = keras.layers.Input(shape=(height, width, channels))

encoded_l = original_model(left_input)
encoded_r = original_model(right_input)

L1_layer = keras.layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
merge_layer = L1_layer([encoded_l, encoded_r])

prediction = keras.layers.Dense(1, activation="sigmoid")(merge_layer)

original_siamese_model_2 = keras.models.Model([left_input, right_input], outputs=prediction)

In [211]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.001,
                         "momentum": 0.5,
                         "otimizer": "SGD",
                         "loss_function": "binary_crossentropy",
                         "epochs": 200,
                         "architecture": "Original - 90k"})

In [212]:
config = wandb.config

In [213]:
original_siamese_model_2.compile(loss=config.loss_function,
                               optimizer=keras.optimizers.SGD(learning_rate=config.learning_rate, momentum=config.momentum), metrics=["accuracy"])

In [214]:
def scheduler(epoch, lr):
    new_lr =  lr * 0.99
    print(f"learning rate scheduled to {new_lr}")
    return new_lr

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="Architecture_1_Checkpoints/Original_90k",
    save_weights_only=True,
    monitor="val_accuracy",
    save_best_only=True
)

In [215]:
history_original_siamese_model_2 = original_siamese_model_2.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.LearningRateScheduler(scheduler),
                                                                                                                                         model_checkpoint_callback, WandbCallback()])

Epoch 1/200
learning rate scheduled to 0.0009900000470224768
610/610 [==============================] - 45s 64ms/step - loss: 1507.0350 - accuracy: 0.5532 - val_loss: 1503.3882 - val_accuracy: 0.5779
Epoch 2/200
learning rate scheduled to 0.000980100086890161
610/610 [==============================] - 42s 63ms/step - loss: 1499.8073 - accuracy: 0.5696 - val_loss: 1496.2098 - val_accuracy: 0.5619
Epoch 3/200
learning rate scheduled to 0.0009702991275116801
610/610 [==============================] - 42s 62ms/step - loss: 1492.6814 - accuracy: 0.5683 - val_loss: 1489.1345 - val_accuracy: 0.5802
Epoch 4/200
learning rate scheduled to 0.0009605961316265165
610/610 [==============================] - 42s 63ms/step - loss: 1485.6611 - accuracy: 0.5803 - val_loss: 1482.1699 - val_accuracy: 0.5906
Epoch 5/200
learning rate scheduled to 0.0009509901772253215
610/610 [==============================] - 42s 63ms/step - loss: 1478.7444 - accuracy: 0.5917 - val_loss: 1475.3059 - val_accuracy: 0.5928
E

In [216]:
loss, accuracy = original_siamese_model_2.evaluate(val_dataset)
wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)})

153/153 [==============================] - 7s 20ms/step - loss: 994.0361 - accuracy: 0.8654


In [217]:
anchor_images_path = "npz_datasets/test_pairs_224_224/anchor"
positive_images_path = "npz_datasets/test_pairs_224_224/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_path, positive_images_path, height, width, rgb=True)

In [218]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, original_siamese_model_2)

In [219]:
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()

## Third Run - 150k Pairs

In [4]:
anchor_images_path = "npz_datasets/pairs_150k_224_224/anchor"
positive_images_path = "npz_datasets/pairs_150k_224_224/positive"
width, height, channels = 105, 105, 3
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=True)

In [5]:
original_model = get_original_model(height,width,channels)
original_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 105, 105, 3)]     0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 96, 96, 64)        19264     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 64)        0         
_________________________________________________________________
Conv2 (Conv2D)               (None, 42, 42, 128)       401536    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 21, 21, 128)       0         
_________________________________________________________________
Conv3 (Conv2D)               (None, 18, 18, 128)       262272    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 9, 128)         0     

In [6]:
left_input = keras.layers.Input(shape=(height, width, channels))
right_input = keras.layers.Input(shape=(height, width, channels))

encoded_l = original_model(left_input)
encoded_r = original_model(right_input)

L1_layer = keras.layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
merge_layer = L1_layer([encoded_l, encoded_r])

prediction = keras.layers.Dense(1, activation="sigmoid")(merge_layer)

original_siamese_model_3 = keras.models.Model([left_input, right_input], outputs=prediction)

In [7]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.001,
                         "momentum": 0.5,
                         "otimizer": "SGD",
                         "loss_function": "binary_crossentropy",
                         "epochs": 200,
                         "architecture": "Original - 150k"})

wandb: Currently logged in as: schauppi (use `wandb login --relogin` to force relogin)


In [8]:
config = wandb.config

In [9]:
original_siamese_model_3.compile(loss=config.loss_function,
                                 optimizer=keras.optimizers.SGD(learning_rate=config.learning_rate, momentum=config.momentum), metrics=["accuracy"])

In [10]:
def scheduler(epoch, lr):
    new_lr =  lr * 0.99
    print(f"learning rate scheduled to {new_lr}")
    return new_lr

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="Architecture_1_Checkpoints/Original_150k",
    save_weights_only=True,
    monitor="val_accuracy",
    save_best_only=True
)

In [11]:
history_original_siamese_model_3 = original_siamese_model_3.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.LearningRateScheduler(scheduler),
                                                                                                                                             model_checkpoint_callback, WandbCallback()])

Epoch 1/200
learning rate scheduled to 0.0009900000470224768
1014/1014 [==============================] - 78s 67ms/step - loss: 1504.4772 - accuracy: 0.5708 - val_loss: 1498.4337 - val_accuracy: 0.5637
Epoch 2/200
learning rate scheduled to 0.000980100086890161
1014/1014 [==============================] - 71s 64ms/step - loss: 1492.4955 - accuracy: 0.5783 - val_loss: 1486.5610 - val_accuracy: 0.5823
Epoch 3/200
learning rate scheduled to 0.0009702991275116801
1014/1014 [==============================] - 72s 65ms/step - loss: 1480.7300 - accuracy: 0.5948 - val_loss: 1474.8993 - val_accuracy: 0.6120
Epoch 4/200
learning rate scheduled to 0.0009605961316265165
1014/1014 [==============================] - 73s 66ms/step - loss: 1469.1669 - accuracy: 0.6262 - val_loss: 1463.4354 - val_accuracy: 0.6393
Epoch 5/200
learning rate scheduled to 0.0009509901772253215
1014/1014 [==============================] - 72s 65ms/step - loss: 1457.7999 - accuracy: 0.6574 - val_loss: 1452.1571 - val_accuracy

In [14]:
loss, accuracy = original_siamese_model_3.evaluate(val_dataset)
wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)})

254/254 [==============================] - 11s 21ms/step - loss: 753.4119 - accuracy: 0.8999


In [12]:
anchor_images_path = "npz_datasets/test_pairs_224_224/anchor"
positive_images_path = "npz_datasets/test_pairs_224_224/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_path, positive_images_path, height, width, rgb=True)

In [13]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, original_siamese_model_3)

In [16]:
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()

## Fourth Run - 30k Pairs gray

In [20]:
anchor_images_path = "npz_datasets/pairs_30k_224_224/anchor"
positive_images_path = "npz_datasets/pairs_30k_224_224/positive"
width, height, channels = 105, 105, 1
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=False)

In [21]:
original_model = get_original_model(height,width,channels)
original_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 105, 105, 1)]     0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 96, 96, 64)        6464      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
Conv2 (Conv2D)               (None, 42, 42, 128)       401536    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 21, 21, 128)       0         
_________________________________________________________________
Conv3 (Conv2D)               (None, 18, 18, 128)       262272    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 9, 9, 128)         0   

In [22]:
left_input = keras.layers.Input(shape=(height, width, channels))
right_input = keras.layers.Input(shape=(height, width, channels))

encoded_l = original_model(left_input)
encoded_r = original_model(right_input)

L1_layer = keras.layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
merge_layer = L1_layer([encoded_l, encoded_r])

prediction = keras.layers.Dense(1, activation="sigmoid")(merge_layer)

original_siamese_model_4 = keras.models.Model([left_input, right_input], outputs=prediction)

In [12]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.001,
                         "momentum": 0.5,
                         "otimizer": "SGD",
                         "loss_function": "binary_crossentropy",
                         "epochs": 200,
                         "architecture": "Original - 30k - Grayscale"})

wandb: Currently logged in as: schauppi (use `wandb login --relogin` to force relogin)


In [13]:
config = wandb.config

In [23]:
original_siamese_model_4.compile(loss=config.loss_function,
                                 optimizer=keras.optimizers.SGD(learning_rate=config.learning_rate, momentum=config.momentum), metrics=["accuracy"])

In [25]:
def scheduler(epoch, lr):
    new_lr =  lr * 0.99
    print(f"learning rate scheduled to {new_lr}")
    return new_lr

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="Architecture_1_Checkpoints/Original_30k_Gray",
    save_weights_only=True,
    monitor="val_accuracy",
    save_best_only=True
)

In [26]:
history_original_siamese_model_4 = original_siamese_model_4.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.LearningRateScheduler(scheduler),
                                                                                                                                             model_checkpoint_callback, WandbCallback()])

Epoch 1/200
learning rate scheduled to 0.0009900000470224768
203/203 [==============================] - 12s 55ms/step - loss: 1509.4403 - accuracy: 0.5015 - val_loss: 1508.2151 - val_accuracy: 0.5015
Epoch 2/200
learning rate scheduled to 0.000980100086890161
203/203 [==============================] - 11s 52ms/step - loss: 1507.0282 - accuracy: 0.5073 - val_loss: 1505.8185 - val_accuracy: 0.5021
Epoch 3/200
learning rate scheduled to 0.0009702991275116801
203/203 [==============================] - 11s 53ms/step - loss: 1504.6437 - accuracy: 0.5033 - val_loss: 1503.4476 - val_accuracy: 0.4926
Epoch 4/200
learning rate scheduled to 0.0009605961316265165
203/203 [==============================] - 11s 52ms/step - loss: 1502.2870 - accuracy: 0.5165 - val_loss: 1501.1044 - val_accuracy: 0.5160
Epoch 5/200
learning rate scheduled to 0.0009509901772253215
203/203 [==============================] - 11s 52ms/step - loss: 1499.9572 - accuracy: 0.5246 - val_loss: 1498.7887 - val_accuracy: 0.5206
E

In [27]:
loss, accuracy = original_siamese_model_4.evaluate(val_dataset)
wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)})

51/51 [==============================] - 2s 18ms/step - loss: 1314.2748 - accuracy: 0.6682


In [28]:
anchor_images_path = "npz_datasets/test_pairs_224_224/anchor"
positive_images_path = "npz_datasets/test_pairs_224_224/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_path, positive_images_path, height, width, rgb=False)

In [29]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, original_siamese_model_4)

In [30]:
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()

## Fifth Run - 90k Pairs gray

In [5]:
anchor_images_path = "npz_datasets/pairs_90k_224_224/anchor"
positive_images_path = "npz_datasets/pairs_90k_224_224/positive"
width, height, channels = 105, 105, 1
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=False)

In [6]:
original_model = get_original_model(height,width,channels)
original_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 105, 105, 1)]     0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 96, 96, 64)        6464      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 64)        0         
_________________________________________________________________
Conv2 (Conv2D)               (None, 42, 42, 128)       401536    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 21, 21, 128)       0         
_________________________________________________________________
Conv3 (Conv2D)               (None, 18, 18, 128)       262272    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 9, 128)         0     

In [7]:
left_input = keras.layers.Input(shape=(height, width, channels))
right_input = keras.layers.Input(shape=(height, width, channels))

encoded_l = original_model(left_input)
encoded_r = original_model(right_input)

L1_layer = keras.layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
merge_layer = L1_layer([encoded_l, encoded_r])

prediction = keras.layers.Dense(1, activation="sigmoid")(merge_layer)

original_siamese_model_5 = keras.models.Model([left_input, right_input], outputs=prediction)

In [8]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.001,
                         "momentum": 0.5,
                         "otimizer": "SGD",
                         "loss_function": "binary_crossentropy",
                         "epochs": 200,
                         "architecture": "Original - 90k - Grayscale"})

wandb: Currently logged in as: schauppi (use `wandb login --relogin` to force relogin)


In [9]:
config = wandb.config

In [10]:
original_siamese_model_5.compile(loss=config.loss_function,
                                 optimizer=keras.optimizers.SGD(learning_rate=config.learning_rate, momentum=config.momentum), metrics=["accuracy"])

In [11]:
def scheduler(epoch, lr):
    new_lr =  lr * 0.99
    print(f"learning rate scheduled to {new_lr}")
    return new_lr

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="Architecture_1_Checkpoints/Original_90k_Gray",
    save_weights_only=True,
    monitor="val_accuracy",
    save_best_only=True
)

In [12]:
history_original_siamese_model_5 = original_siamese_model_5.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.LearningRateScheduler(scheduler),
                                                                                                                                             model_checkpoint_callback, WandbCallback()])

Epoch 1/200
learning rate scheduled to 0.0009900000470224768
610/610 [==============================] - 38s 53ms/step - loss: 1507.2825 - accuracy: 0.5475 - val_loss: 1503.6361 - val_accuracy: 0.5430
Epoch 2/200
learning rate scheduled to 0.000980100086890161
610/610 [==============================] - 34s 53ms/step - loss: 1500.0554 - accuracy: 0.5328 - val_loss: 1496.4590 - val_accuracy: 0.5303
Epoch 3/200
learning rate scheduled to 0.0009702991275116801
610/610 [==============================] - 34s 53ms/step - loss: 1492.9326 - accuracy: 0.5362 - val_loss: 1489.3882 - val_accuracy: 0.5388
Epoch 4/200
learning rate scheduled to 0.0009605961316265165
610/610 [==============================] - 34s 53ms/step - loss: 1485.9120 - accuracy: 0.5510 - val_loss: 1482.4197 - val_accuracy: 0.5687
Epoch 5/200
learning rate scheduled to 0.0009509901772253215
610/610 [==============================] - 34s 53ms/step - loss: 1478.9958 - accuracy: 0.5724 - val_loss: 1475.5557 - val_accuracy: 0.5787
E

In [13]:
loss, accuracy = original_siamese_model_5.evaluate(val_dataset)
wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)})

153/153 [==============================] - 5s 18ms/step - loss: 994.3560 - accuracy: 0.7681


In [14]:
anchor_images_path = "npz_datasets/test_pairs_224_224/anchor"
positive_images_path = "npz_datasets/test_pairs_224_224/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_path, positive_images_path, height, width, rgb=False)

In [15]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, original_siamese_model_5)

In [16]:
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()

## Sixth Run - 150k Pairs Gray

In [18]:
anchor_images_path = "npz_datasets/pairs_150k_224_224/anchor"
positive_images_path = "npz_datasets/pairs_150k_224_224/positive"
width, height, channels = 105, 105, 1
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=False)

In [19]:
original_model = get_original_model(height,width,channels)
original_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 105, 105, 1)]     0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 96, 96, 64)        6464      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
Conv2 (Conv2D)               (None, 42, 42, 128)       401536    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 21, 21, 128)       0         
_________________________________________________________________
Conv3 (Conv2D)               (None, 18, 18, 128)       262272    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 9, 9, 128)         0   

In [20]:
left_input = keras.layers.Input(shape=(height, width, channels))
right_input = keras.layers.Input(shape=(height, width, channels))

encoded_l = original_model(left_input)
encoded_r = original_model(right_input)

L1_layer = keras.layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
merge_layer = L1_layer([encoded_l, encoded_r])

prediction = keras.layers.Dense(1, activation="sigmoid")(merge_layer)

original_siamese_model_6 = keras.models.Model([left_input, right_input], outputs=prediction)

In [21]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.001,
                         "momentum": 0.5,
                         "otimizer": "SGD",
                         "loss_function": "binary_crossentropy",
                         "epochs": 200,
                         "architecture": "Original - 150k - Grayscale"})

In [22]:
config = wandb.config

In [23]:
original_siamese_model_6.compile(loss=config.loss_function,
                                 optimizer=keras.optimizers.SGD(learning_rate=config.learning_rate, momentum=config.momentum), metrics=["accuracy"])

In [24]:
def scheduler(epoch, lr):
    new_lr =  lr * 0.99
    print(f"learning rate scheduled to {new_lr}")
    return new_lr

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="Architecture_1_Checkpoints/Original_150k_Gray",
    save_weights_only=True,
    monitor="val_accuracy",
    save_best_only=True
)

In [25]:
history_original_siamese_model_6 = original_siamese_model_6.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.LearningRateScheduler(scheduler),
                                                                                                                                             model_checkpoint_callback, WandbCallback()])

Epoch 1/200
learning rate scheduled to 0.0009900000470224768
1014/1014 [==============================] - 63s 58ms/step - loss: 1504.8086 - accuracy: 0.5364 - val_loss: 1498.7686 - val_accuracy: 0.5598
Epoch 2/200
learning rate scheduled to 0.000980100086890161
1014/1014 [==============================] - 62s 57ms/step - loss: 1492.8362 - accuracy: 0.5633 - val_loss: 1486.9006 - val_accuracy: 0.5545
Epoch 3/200
learning rate scheduled to 0.0009702991275116801
1014/1014 [==============================] - 58s 54ms/step - loss: 1481.0712 - accuracy: 0.5507 - val_loss: 1475.2389 - val_accuracy: 0.5629
Epoch 4/200
learning rate scheduled to 0.0009605961316265165
1014/1014 [==============================] - 57s 53ms/step - loss: 1469.5118 - accuracy: 0.5697 - val_loss: 1463.7858 - val_accuracy: 0.5808
Epoch 5/200
learning rate scheduled to 0.0009509901772253215
1014/1014 [==============================] - 57s 53ms/step - loss: 1458.1604 - accuracy: 0.5860 - val_loss: 1452.5353 - val_accuracy

In [26]:
loss, accuracy = original_siamese_model_6.evaluate(val_dataset)
wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)})

254/254 [==============================] - 8s 19ms/step - loss: 753.7661 - accuracy: 0.7866


In [27]:
anchor_images_path = "npz_datasets/test_pairs_224_224/anchor"
positive_images_path = "npz_datasets/test_pairs_224_224/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_path, positive_images_path, height, width, rgb=False)

In [28]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, original_siamese_model_6)

In [30]:
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()

# Adapted Model

In [3]:
mixed_precision.set_global_policy('mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce RTX 3090, compute capability 8.6


In [4]:
def get_adapted_model(height, width, channels):

    input = keras.layers.Input(shape=(height, width, channels))

    x = keras.layers.Conv2D(96, kernel_size=5, strides=2, activation='relu')(input)
    x = keras.layers.MaxPooling2D(3, strides=2)(x)

    x = keras.layers.Conv2D(256, kernel_size=3, strides=1, padding="valid", activation='relu')(x)
    x = keras.layers.MaxPooling2D(3, strides=2)(x)

    x = keras.layers.Conv2D(384, kernel_size=3, strides=1, padding="valid", activation='relu')(x)
    x = keras.layers.Conv2D(384, kernel_size=3, strides=1, padding="valid", activation='relu')(x)
    x = keras.layers.Conv2D(256, kernel_size=3, strides=1, padding="valid",  activation='relu')(x)
    x = keras.layers.MaxPooling2D(3, strides=2)(x)

    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(1024, activation='relu')(x)
    output = keras.layers.Dense(1024, activation='relu')(x)

    model = keras.models.Model(input, output)

    return model

## First Run - 30k Pairs

In [5]:
anchor_images_path = "npz_datasets/pairs_30k_224_224/anchor"
positive_images_path = "npz_datasets/pairs_30k_224_224/positive"
width, height, channels = 113, 113, 3
batch_size = 256
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=True)

In [6]:
adapted_model = get_adapted_model(height,width,channels)

In [7]:
adapted_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 113, 113, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 55, 55, 96)        7296      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 25, 25, 256)       221440    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 384)       885120    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 384)         132748

In [9]:
left_input = keras.layers.Input(shape=(height, width, channels))
right_input = keras.layers.Input(shape=(height, width, channels))

encoded_l = adapted_model(left_input)
encoded_r = adapted_model(right_input)

L1_layer = keras.layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
merge_layer = L1_layer([encoded_l, encoded_r])

prediction = keras.layers.Dense(1, activation="sigmoid")(merge_layer)

adapted_siamese_model = keras.models.Model([left_input, right_input], outputs=prediction)

In [10]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.01,
                         "momentum": 0.9,
                         "weight_decay": 0.0005,
                         "otimizer": "SGDW",
                         "loss_function": "binary_crossentropy",
                         "epochs": 150,
                         "architecture": "Adapted - 30k"})

wandb: Currently logged in as: schauppi (use `wandb login --relogin` to force relogin)


In [11]:
config = wandb.config

In [12]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="Architecture_1_Checkpoints/Adapted_30k",
    save_weights_only=True,
    monitor="val_accuracy",
    save_best_only=True
)

In [13]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=5, verbose=1)

In [14]:
adapted_siamese_model.compile(loss=config.loss_function,
                               optimizer=keras.optimizers.SGD(learning_rate=config.learning_rate, momentum=config.momentum), metrics=["accuracy"])

In [15]:
history_adapted_siamese_model = adapted_siamese_model.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[model_checkpoint_callback, WandbCallback(), reduce_lr])

Epoch 1/150
188/188 [==============================] - 15s 49ms/step - loss: 0.6763 - accuracy: 0.5548 - val_loss: 0.6513 - val_accuracy: 0.6459
Epoch 2/150
188/188 [==============================] - 10s 46ms/step - loss: 0.6297 - accuracy: 0.6505 - val_loss: 0.6287 - val_accuracy: 0.6478
Epoch 3/150
188/188 [==============================] - 10s 46ms/step - loss: 0.5947 - accuracy: 0.6785 - val_loss: 0.5679 - val_accuracy: 0.7034
Epoch 4/150
188/188 [==============================] - 10s 45ms/step - loss: 0.5649 - accuracy: 0.7073 - val_loss: 0.6107 - val_accuracy: 0.6682
Epoch 5/150
188/188 [==============================] - 10s 46ms/step - loss: 0.5299 - accuracy: 0.7363 - val_loss: 0.4959 - val_accuracy: 0.7515
Epoch 6/150
188/188 [==============================] - 10s 46ms/step - loss: 0.5098 - accuracy: 0.7543 - val_loss: 0.4750 - val_accuracy: 0.7781
Epoch 7/150
188/188 [==============================] - 10s 45ms/step - loss: 0.4700 - accuracy: 0.7805 - val_loss: 0.4515 - val_ac

In [16]:
loss, accuracy = adapted_siamese_model.evaluate(val_dataset)
wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)})

47/47 [==============================] - 2s 15ms/step - loss: 0.0016 - accuracy: 1.0000


In [17]:
anchor_images_path = "npz_datasets/test_pairs_224_224/anchor"
positive_images_path = "npz_datasets/test_pairs_224_224/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_path, positive_images_path, height, width, rgb=True)

In [21]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, adapted_siamese_model)

In [22]:
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()

## Second Run - 90k Pairs

In [35]:
anchor_images_path = "npz_datasets/pairs_90k_224_224/anchor"
positive_images_path = "npz_datasets/pairs_90k_224_224/positive"
width, height, channels = 113, 113, 3
batch_size = 256
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=True)

In [36]:
adapted_model = get_adapted_model(height,width,channels)

In [37]:
adapted_model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 113, 113, 3)]     0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 55, 55, 96)        7296      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 25, 25, 256)       221440    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 10, 10, 384)       885120    
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 8, 8, 384)         1327

In [38]:
left_input = keras.layers.Input(shape=(height, width, channels))
right_input = keras.layers.Input(shape=(height, width, channels))

encoded_l = adapted_model(left_input)
encoded_r = adapted_model(right_input)

L1_layer = keras.layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
merge_layer = L1_layer([encoded_l, encoded_r])

prediction = keras.layers.Dense(1, activation="sigmoid")(merge_layer)

adapted_siamese_model_2 = keras.models.Model([left_input, right_input], outputs=prediction)

In [39]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.01,
                         "momentum": 0.9,
                         "weight_decay": 0.0005,
                         "otimizer": "SGDW",
                         "loss_function": "binary_crossentropy",
                         "epochs": 150,
                         "architecture": "Adapted - 90k"})

In [40]:
config = wandb.config

In [41]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="Architecture_1_Checkpoints/Adapted_90k",
    save_weights_only=True,
    monitor="val_accuracy",
    save_best_only=True
)

In [42]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                                 patience=5, verbose=1)

In [43]:
adapted_siamese_model_2.compile(loss=config.loss_function,
                              optimizer=keras.optimizers.SGD(learning_rate=config.learning_rate, momentum=config.momentum), metrics=["accuracy"])

In [44]:
history_adapted_siamese_model_2 = adapted_siamese_model_2.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[model_checkpoint_callback, WandbCallback(), reduce_lr])

Epoch 1/150
566/566 [==============================] - 34s 48ms/step - loss: 0.6273 - accuracy: 0.6376 - val_loss: 0.5553 - val_accuracy: 0.7143
Epoch 2/150
566/566 [==============================] - 31s 47ms/step - loss: 0.5280 - accuracy: 0.7400 - val_loss: 0.4869 - val_accuracy: 0.7668
Epoch 3/150
566/566 [==============================] - 30s 46ms/step - loss: 0.4671 - accuracy: 0.7814 - val_loss: 0.4576 - val_accuracy: 0.7926
Epoch 4/150
566/566 [==============================] - 30s 46ms/step - loss: 0.4440 - accuracy: 0.7930 - val_loss: 0.4209 - val_accuracy: 0.8089
Epoch 5/150
566/566 [==============================] - 30s 47ms/step - loss: 0.4172 - accuracy: 0.8091 - val_loss: 0.4264 - val_accuracy: 0.8061
Epoch 6/150
566/566 [==============================] - 30s 46ms/step - loss: 0.4006 - accuracy: 0.8191 - val_loss: 0.3807 - val_accuracy: 0.8349
Epoch 7/150
566/566 [==============================] - 30s 46ms/step - loss: 0.3854 - accuracy: 0.8292 - val_loss: 0.3889 - val_ac

In [45]:
loss, accuracy = adapted_siamese_model_2.evaluate(val_dataset)
wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)})

142/142 [==============================] - 6s 15ms/step - loss: 7.1380e-04 - accuracy: 0.9999


In [46]:
anchor_images_path = "npz_datasets/test_pairs_224_224/anchor"
positive_images_path = "npz_datasets/test_pairs_224_224/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_path, positive_images_path, height, width, rgb=True)

In [47]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, adapted_siamese_model_2)

In [49]:
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()

## Thirt Run - 150k Pairs

In [52]:
anchor_images_path = "npz_datasets/pairs_150k_224_224/anchor"
positive_images_path = "npz_datasets/pairs_150k_224_224/positive"
width, height, channels = 113, 113, 3
batch_size = 256
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=True)

In [53]:
adapted_model = get_adapted_model(height,width,channels)

In [54]:
adapted_model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 113, 113, 3)]     0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 55, 55, 96)        7296      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 25, 25, 256)       221440    
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 10, 10, 384)       885120    
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 8, 8, 384)         1327

In [55]:
left_input = keras.layers.Input(shape=(height, width, channels))
right_input = keras.layers.Input(shape=(height, width, channels))

encoded_l = adapted_model(left_input)
encoded_r = adapted_model(right_input)

L1_layer = keras.layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
merge_layer = L1_layer([encoded_l, encoded_r])

prediction = keras.layers.Dense(1, activation="sigmoid")(merge_layer)

adapted_siamese_model_3 = keras.models.Model([left_input, right_input], outputs=prediction)

In [56]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.01,
                         "momentum": 0.9,
                         "weight_decay": 0.0005,
                         "otimizer": "SGDW",
                         "loss_function": "binary_crossentropy",
                         "epochs": 150,
                         "architecture": "Adapted - 150k"})

In [57]:
config = wandb.config

In [58]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="Architecture_1_Checkpoints/Adapted_150k",
    save_weights_only=True,
    monitor="val_accuracy",
    save_best_only=True
)

In [59]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                                 patience=5, verbose=1)

In [60]:
adapted_siamese_model_3.compile(loss=config.loss_function,
                                optimizer=keras.optimizers.SGD(learning_rate=config.learning_rate, momentum=config.momentum), metrics=["accuracy"])

In [61]:
history_adapted_siamese_model_3 = adapted_siamese_model_3.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[model_checkpoint_callback, WandbCallback(), reduce_lr])

Epoch 1/150
942/942 [==============================] - 53s 47ms/step - loss: 0.5931 - accuracy: 0.6747 - val_loss: 0.4841 - val_accuracy: 0.7780
Epoch 2/150
942/942 [==============================] - 50s 46ms/step - loss: 0.4699 - accuracy: 0.7790 - val_loss: 0.5124 - val_accuracy: 0.7554
Epoch 3/150
942/942 [==============================] - 50s 46ms/step - loss: 0.4243 - accuracy: 0.8053 - val_loss: 0.4127 - val_accuracy: 0.8153
Epoch 4/150
942/942 [==============================] - 49s 45ms/step - loss: 0.4053 - accuracy: 0.8154 - val_loss: 0.3818 - val_accuracy: 0.8325
Epoch 5/150
942/942 [==============================] - 50s 46ms/step - loss: 0.3758 - accuracy: 0.8333 - val_loss: 0.3777 - val_accuracy: 0.8336
Epoch 6/150
942/942 [==============================] - 49s 45ms/step - loss: 0.3657 - accuracy: 0.8387 - val_loss: 0.3677 - val_accuracy: 0.8369
Epoch 7/150
942/942 [==============================] - 49s 46ms/step - loss: 0.3578 - accuracy: 0.8430 - val_loss: 0.3521 - val_ac

In [62]:
loss, accuracy = adapted_siamese_model_3.evaluate(val_dataset)
wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)})

236/236 [==============================] - 9s 15ms/step - loss: 2.6775e-04 - accuracy: 1.0000


In [63]:
anchor_images_path = "npz_datasets/test_pairs_224_224/anchor"
positive_images_path = "npz_datasets/test_pairs_224_224/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_path, positive_images_path, height, width, rgb=True)

In [64]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, adapted_siamese_model_3)

In [65]:
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()

## Forth Run - 30k pairs gray

In [11]:
anchor_images_path = "npz_datasets/pairs_30k_224_224/anchor"
positive_images_path = "npz_datasets/pairs_30k_224_224/positive"
width, height, channels = 113, 113, 1
batch_size = 256
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=False)

In [8]:
adapted_model = get_adapted_model(height,width,channels)

In [9]:
adapted_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 113, 113, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 55, 55, 96)        2496      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 25, 25, 256)       221440    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 384)       885120    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 384)         132748

In [10]:
left_input = keras.layers.Input(shape=(height, width, channels))
right_input = keras.layers.Input(shape=(height, width, channels))

encoded_l = adapted_model(left_input)
encoded_r = adapted_model(right_input)

L1_layer = keras.layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
merge_layer = L1_layer([encoded_l, encoded_r])

prediction = keras.layers.Dense(1, activation="sigmoid")(merge_layer)

adapted_siamese_model_4 = keras.models.Model([left_input, right_input], outputs=prediction)

In [12]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.01,
                         "momentum": 0.9,
                         "weight_decay": 0.0005,
                         "otimizer": "SGDW",
                         "loss_function": "binary_crossentropy",
                         "epochs": 150,
                         "architecture": "Adapted - 30k - Grayscale"})

wandb: Currently logged in as: schauppi (use `wandb login --relogin` to force relogin)


In [13]:
config = wandb.config

In [14]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="Architecture_1_Checkpoints/Adapted_30k_Gray",
    save_weights_only=True,
    monitor="val_accuracy",
    save_best_only=True
)

In [15]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                                 patience=5, verbose=1)

In [16]:
adapted_siamese_model_4.compile(loss=config.loss_function,
                                optimizer=keras.optimizers.SGD(learning_rate=config.learning_rate, momentum=config.momentum), metrics=["accuracy"])

In [17]:
history_adapted_siamese_model_4 = adapted_siamese_model_4.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[model_checkpoint_callback, WandbCallback(), reduce_lr])

Epoch 1/150
188/188 [==============================] - 12s 42ms/step - loss: 0.6856 - accuracy: 0.5652 - val_loss: 0.6586 - val_accuracy: 0.6088
Epoch 2/150
188/188 [==============================] - 8s 38ms/step - loss: 0.6382 - accuracy: 0.6433 - val_loss: 0.6179 - val_accuracy: 0.6637
Epoch 3/150
188/188 [==============================] - 8s 38ms/step - loss: 0.6094 - accuracy: 0.6641 - val_loss: 0.6096 - val_accuracy: 0.6614
Epoch 4/150
188/188 [==============================] - 8s 38ms/step - loss: 0.5959 - accuracy: 0.6775 - val_loss: 0.5853 - val_accuracy: 0.6733
Epoch 5/150
188/188 [==============================] - 8s 38ms/step - loss: 0.5885 - accuracy: 0.6817 - val_loss: 0.5849 - val_accuracy: 0.6921
Epoch 6/150
188/188 [==============================] - 8s 38ms/step - loss: 0.5817 - accuracy: 0.6884 - val_loss: 0.5759 - val_accuracy: 0.6899
Epoch 7/150
188/188 [==============================] - 8s 38ms/step - loss: 0.5775 - accuracy: 0.6888 - val_loss: 0.5722 - val_accuracy

In [18]:
loss, accuracy = adapted_siamese_model_4.evaluate(val_dataset)
wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)})

47/47 [==============================] - 1s 10ms/step - loss: 3.0935e-04 - accuracy: 1.0000


In [19]:
anchor_images_path = "npz_datasets/test_pairs_224_224/anchor"
positive_images_path = "npz_datasets/test_pairs_224_224/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_path, positive_images_path, height, width, rgb=False)

In [20]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, adapted_siamese_model_4)

In [22]:
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()

## Fifth Run - 90k Pairs gray

In [24]:
anchor_images_path = "npz_datasets/pairs_90k_224_224/anchor"
positive_images_path = "npz_datasets/pairs_90k_224_224/positive"
width, height, channels = 113, 113, 1
batch_size = 256
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=False)

In [25]:
adapted_model = get_adapted_model(height,width,channels)

In [26]:
adapted_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 113, 113, 1)]     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 55, 55, 96)        2496      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 25, 25, 256)       221440    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 10, 10, 384)       885120    
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 8, 8, 384)         1327

In [27]:
left_input = keras.layers.Input(shape=(height, width, channels))
right_input = keras.layers.Input(shape=(height, width, channels))

encoded_l = adapted_model(left_input)
encoded_r = adapted_model(right_input)

L1_layer = keras.layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
merge_layer = L1_layer([encoded_l, encoded_r])

prediction = keras.layers.Dense(1, activation="sigmoid")(merge_layer)

adapted_siamese_model_5 = keras.models.Model([left_input, right_input], outputs=prediction)

In [28]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.01,
                         "momentum": 0.9,
                         "weight_decay": 0.0005,
                         "otimizer": "SGDW",
                         "loss_function": "binary_crossentropy",
                         "epochs": 150,
                         "architecture": "Adapted - 90k - Grayscale"})

In [29]:
config = wandb.config

In [30]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="Architecture_1_Checkpoints/Adapted_90k_Gray",
    save_weights_only=True,
    monitor="val_accuracy",
    save_best_only=True
)

In [31]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                                 patience=5, verbose=1)

In [32]:
adapted_siamese_model_5.compile(loss=config.loss_function,
                                optimizer=keras.optimizers.SGD(learning_rate=config.learning_rate, momentum=config.momentum), metrics=["accuracy"])

In [33]:
history_adapted_siamese_model_5 = adapted_siamese_model_5.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[model_checkpoint_callback, WandbCallback(), reduce_lr])

Epoch 1/150
566/566 [==============================] - 25s 39ms/step - loss: 0.6450 - accuracy: 0.6156 - val_loss: 0.5977 - val_accuracy: 0.6813
Epoch 2/150
566/566 [==============================] - 23s 38ms/step - loss: 0.5958 - accuracy: 0.6771 - val_loss: 0.5780 - val_accuracy: 0.6915
Epoch 3/150
566/566 [==============================] - 23s 38ms/step - loss: 0.5786 - accuracy: 0.6886 - val_loss: 0.5694 - val_accuracy: 0.6902
Epoch 4/150
566/566 [==============================] - 23s 38ms/step - loss: 0.5570 - accuracy: 0.7063 - val_loss: 0.5591 - val_accuracy: 0.7059
Epoch 5/150
566/566 [==============================] - 25s 41ms/step - loss: 0.5383 - accuracy: 0.7230 - val_loss: 0.5343 - val_accuracy: 0.7245
Epoch 6/150
566/566 [==============================] - 24s 38ms/step - loss: 0.5229 - accuracy: 0.7320 - val_loss: 0.5523 - val_accuracy: 0.7165
Epoch 7/150
566/566 [==============================] - 23s 38ms/step - loss: 0.5127 - accuracy: 0.7375 - val_loss: 0.5059 - val_ac

In [34]:
loss, accuracy = adapted_siamese_model_5.evaluate(val_dataset)
wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)})

142/142 [==============================] - 4s 10ms/step - loss: 2.4646e-04 - accuracy: 1.0000


In [35]:
anchor_images_path = "npz_datasets/test_pairs_224_224/anchor"
positive_images_path = "npz_datasets/test_pairs_224_224/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_path, positive_images_path, height, width, rgb=False)

In [36]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, adapted_siamese_model_5)

In [38]:
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()

## Sixth Run - 150k Gray

In [40]:
anchor_images_path = "npz_datasets/pairs_150k_224_224/anchor"
positive_images_path = "npz_datasets/pairs_150k_224_224/positive"
width, height, channels = 113, 113, 1
batch_size = 256
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=False)

In [41]:
adapted_model = get_adapted_model(height,width,channels)

In [42]:
adapted_model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 113, 113, 1)]     0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 55, 55, 96)        2496      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 25, 25, 256)       221440    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 10, 10, 384)       885120    
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 8, 8, 384)         1327

In [43]:
left_input = keras.layers.Input(shape=(height, width, channels))
right_input = keras.layers.Input(shape=(height, width, channels))

encoded_l = adapted_model(left_input)
encoded_r = adapted_model(right_input)

L1_layer = keras.layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
merge_layer = L1_layer([encoded_l, encoded_r])

prediction = keras.layers.Dense(1, activation="sigmoid")(merge_layer)

adapted_siamese_model_6 = keras.models.Model([left_input, right_input], outputs=prediction)

In [44]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.01,
                         "momentum": 0.9,
                         "weight_decay": 0.0005,
                         "otimizer": "SGDW",
                         "loss_function": "binary_crossentropy",
                         "epochs": 150,
                         "architecture": "Adapted - 150k - Grayscale"})

In [45]:
config = wandb.config

In [46]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="Architecture_1_Checkpoints/Adapted_150k_Gray",
    save_weights_only=True,
    monitor="val_accuracy",
    save_best_only=True
)

In [47]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                                 patience=5, verbose=1)

In [48]:
adapted_siamese_model_6.compile(loss=config.loss_function,
                                optimizer=keras.optimizers.SGD(learning_rate=config.learning_rate, momentum=config.momentum), metrics=["accuracy"])

In [49]:
history_adapted_siamese_model_6 = adapted_siamese_model_6.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[model_checkpoint_callback, WandbCallback(), reduce_lr])

Epoch 1/150
942/942 [==============================] - 41s 39ms/step - loss: 0.6255 - accuracy: 0.6493 - val_loss: 0.5861 - val_accuracy: 0.6780
Epoch 2/150
942/942 [==============================] - 40s 38ms/step - loss: 0.5772 - accuracy: 0.6891 - val_loss: 0.5594 - val_accuracy: 0.7042
Epoch 3/150
942/942 [==============================] - 40s 38ms/step - loss: 0.5493 - accuracy: 0.7124 - val_loss: 0.5333 - val_accuracy: 0.7240
Epoch 4/150
942/942 [==============================] - 39s 38ms/step - loss: 0.5280 - accuracy: 0.7290 - val_loss: 0.5062 - val_accuracy: 0.7443
Epoch 5/150
942/942 [==============================] - 39s 38ms/step - loss: 0.5123 - accuracy: 0.7389 - val_loss: 0.5190 - val_accuracy: 0.7311
Epoch 6/150
942/942 [==============================] - 39s 38ms/step - loss: 0.5046 - accuracy: 0.7415 - val_loss: 0.5006 - val_accuracy: 0.7481
Epoch 7/150
942/942 [==============================] - 39s 38ms/step - loss: 0.4957 - accuracy: 0.7479 - val_loss: 0.4830 - val_ac

In [50]:
loss, accuracy = adapted_siamese_model_6.evaluate(val_dataset)
wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)})

236/236 [==============================] - 6s 11ms/step - loss: 2.0901e-04 - accuracy: 1.0000


In [51]:
anchor_images_path = "npz_datasets/test_pairs_224_224/anchor"
positive_images_path = "npz_datasets/test_pairs_224_224/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_path, positive_images_path, height, width, rgb=False)

In [52]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, adapted_siamese_model_6)

In [53]:
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()